In [3]:
import os
import json

from agent.utils.loader import load_prompt, load_processed_data
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, find_dotenv
from langchain_core.prompts.chat import BaseMessage

_ = load_dotenv(find_dotenv())

In [2]:
dataset_name = 'ambig_qa'
mode = "direct"
model = "gpt-4o-mini"
num_samples = 1000
top_p = 1
temperature = 0
processed_data_path = f"../../data/processed_data/{dataset_name}.jsonl"
save_results_path = f"../../output/inference/{model}/{dataset_name}/{mode}/num_samples_{num_samples}_top_p_{top_p}_temperature_{temperature}.jsonl"
prompt = load_prompt(dataset_name=dataset_name, mode=mode)
dataset = load_processed_data(dataset_name=dataset_name, file_path=processed_data_path)
llm = ChatOpenAI(model=model, top_p=top_p, temperature=temperature, base_url="https://api.chsdw.top/v1")

prompt.pretty_print()
print(dataset)

/opt/homebrew/Caskroom/miniconda/base/envs/selfimprove/lib/python3.12/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


================================ Human Message =================================

Q: When did men's figure skating become a summer Olympic sport?

================================== AI Message ==================================

A: 1908

================================ Human Message =================================

Q: When did the all india workers and peasants party came in to existence?

================================== AI Message ==================================

A: November 1925

================================ Human Message =================================

Q: Flight that went down in the hudson river?

================================== AI Message ==================================

A: US Airways Flight 1549

================================ Human Message =================================

Q: Where are most of the world's earthquakes located?

================================== AI Message ==================================

A: Rim of Fire

===============================

In [6]:
from tqdm.asyncio import tqdm
import nest_asyncio
import asyncio

nest_asyncio.apply()

async def direct():
	chain = prompt | llm
	result = []
	if os.path.exists(save_results_path):
	    with open(save_results_path, 'r') as file:
	        for line in file:
	            result.append(json.loads(line))
	
	for idx in tqdm(range(len(result), num_samples)):
		try:
			response: BaseMessage = await chain.ainvoke(input={"question": dataset[idx]["question"]})
			if dataset in ["hotpot_qa", "trivia_qa", "ambig_qa"]:
				result.append({**dataset[idx], "prediction": response.content.split("A: ")[-1]})
			elif dataset in ["gsm9k", "tabmwp", "svamp"]:
				result.append({**dataset[idx], "prediction": response.content.split("Answer: ")[-1]})

		except Exception as e:
			result.append({**dataset[idx], "prediction": "ERROR"})
		
		if idx % 100 == 0:
			with open(save_results_path, 'w') as file:
				for line in result:
					file.write(json.dumps(line) + '\n')
		
asyncio.run(direct())

  0%|          | 0/1000 [00:00<?, ?it/s]/opt/homebrew/Caskroom/miniconda/base/envs/selfimprove/lib/python3.12/inspect.py:3066: RuntimeWarning: coroutine 'direct' was never awaited
  params = OrderedDict((param.name, param) for param in parameters)
  0%|          | 0/1000 [00:01<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: '../../output/inference/gpt-4o-mini/ambig_qa/direct/num_samples_1000_top_p_1_temperature_0.jsonl'